[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: 02 de Octubre del 2025 - hasta las 9:00hs.**
 
<span style="color: red;">**Fecha de entrega intermedia: 23 de Septiembre del 2025 - hasta las 9:00hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 4). 

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido. 

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio. 
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

**El informe no puede superar un máximo de 8 carillas (contando gráficos) o 4 hojas más carátula.** No se corregirán trabajos que no cumplan con esta consigna.

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data.csv')
df.head()

,uAro,vIlg,VtZP,qTUs,bgVs,PdRG,CqGH,bfBD,ZtLG,SNCN,...,hnYp,cWwQ,GkqC,VHPu,KKxk,TNkz,GkcC,KSYM,YpTu,target
0,1.284843,0.676813,-0.445511,-0.025185,-4.546595,0.370719,0.104766,-0.365206,1.155584,0.791519,...,2.052021,-0.256875,-0.961930,4.731984,21.095664,-0.738587,0.728201,1.346252,0.659334,1
1,-0.027713,1.020162,-1.665049,-1.052946,3.144244,2.104813,0.603191,0.180639,1.458030,-0.014031,...,0.891111,-1.328636,-0.848692,-3.989681,7.741832,-0.768312,0.160028,-1.028335,-1.733235,1
2,-0.160656,-1.011420,-0.216399,0.797341,5.672563,7.822798,0.806452,0.386419,-0.854304,0.003736,...,1.411142,2.481312,0.492739,-3.132938,-22.681919,-1.066736,-0.041351,-0.311166,-1.299748,0
3,1.128357,0.341273,-1.669474,0.767591,-4.610520,-8.788584,2.272761,-0.762237,-0.245964,-0.101648,...,1.418474,-0.555290,-0.400534,0.149306,-8.262423,-4.142330,1.256389,-1.252168,-1.597531,0
4,0.573025,1.278193,0.112418,-1.431578,0.589556,1.913852,0.021793,0.533563,0.321761,-0.108064,...,-0.487612,-0.818391,-0.158147,-4.996113,-18.901603,-1.276128,0.318565,-0.592519,1.788781,0


## Ejercicio 1 

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. 

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

### Consideraciones

Mantener la distribución de la variable objetivo, 'target'. Esto es importante porque suponemos que la distribución de los datos se condice con la realidad de los pronósticos.

In [2]:
df_target = df[df['target'] == 1]
print("Proporción de buenos pronósticos", str(len(df_target) / len(df) * 100), "%")

Proporción de buenos pronósticos 29.4 %


In [3]:
import math

def test_train_split(frac):
    
    sample_positive = df[df["target"] == 1].sample(frac=frac,random_state=42).index
    sample_negative = df[df["target"] == 0].sample(frac=frac,random_state=42).index

    train = df.loc[np.append(sample_positive, sample_negative)]    
    test = df.drop(train.index)
    
    #mezclamos porque antes concatenamos positivos con negativos
    train = train.sample(frac=1, random_state=42).reset_index(drop=True)
    
    return train, test

desarrollo, evaluacion = test_train_split(frac=0.9)


## Ejercicio 2


### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

1. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_, y _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.
   
   Reportar el resultado en una tabla similar a:

      <table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <td align="center">Global</td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      </tr>
      </tbody>
      </table>    
  
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.
 
1. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   </tbody></table>

1. ¿Qué conclusiones se pueden sacar de estas tablas?  

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve, roc_curve,auc
def accuracy(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    TP_TN = sum([y_i == y_j for (y_i, y_j) in zip(y_predicted, y_real)]) 
    P_N = len(y_real)
    return TP_TN / P_N

def k_fold(df,model,params ,k = 5):
    # Toma un dataframe, los parametros del modelo y devuelve el vector de predicciones (con accuracys)
    
    metricas={}
    
    metricas["ACCURACY (training)"]=np.zeros(k)
    metricas["AUPRC (training)"]=np.zeros(k)
    metricas["AUCROC (training)"]=np.zeros(k)
    
    metricas["ACCURACY (validacion)"]=np.zeros(k)
    metricas["AUPRC (validacion)"]=np.zeros(k)
    metricas["AUCROC (validacion)"]=np.zeros(k)

    
    predictions = np.zeros(len(df))
    
    k_fold = 0 # va de 0 a 4
    for i in range(0,len(df),len(df)//k):
        #
        #Separacion de datos
        print("fold: ",k_fold+1)
        test = df[i:i+len(df)//k]  #0 90, 90 180, 180 270, 270 360, 360 y 450
        train = df.drop(test.index)
        print("entre",i,":",i+len(df)//k)
        
        #Entrenamiento
        modelo = model(**params)
        x_train = train.iloc[:,0:200].to_numpy()
        y_train = train['target'].to_numpy()
        print("positivos",sum(y_train==1),"negativos",sum(y_train==0))
        x_test = test.iloc[:,0:200].to_numpy()
        
        y_test = test['target'].to_numpy()
        print("positivos",sum(y_test==1),"negativos",sum(y_test==0))
        modelo.fit(x_train, y_train)

        #Prediccion
        y_predict_test = modelo.predict(x_test)
        y_predict_train = modelo.predict(x_train)
        
        #Calculo de Accuracy
        acc_test = accuracy(y_predicted=y_predict_test, y_real=y_test)
        acc_train = accuracy(y_predicted=y_predict_train, y_real=y_train)
        metricas["ACCURACY (validacion)"][k_fold] = acc_test
        metricas["ACCURACY (training)"][k_fold] = acc_train

        #Calculo de AUPRC
        #test
        precision, recall, _ = precision_recall_curve(y_test, y_predict_test)
        auc_score = auc(recall, precision)
        metricas["AUPRC (validacion)"][k_fold]=auc_score
        
        #train
        precision, recall, _ = precision_recall_curve(y_train, y_predict_train)
        auc_score = auc(recall, precision)
        metricas["AUPRC (training)"][k_fold]=auc_score

        #Calculo de AUC ROC
        #test
        fpr, tpr, _ = roc_curve(y_test,y_predict_test )
        roc_auc = auc(fpr, tpr)
        metricas["AUCROC (validacion)"][k_fold]=roc_auc

        #train
        fpr, tpr, _ = roc_curve(y_train,y_predict_train )
        roc_auc = auc(fpr, tpr)
        metricas["AUCROC (training)"][k_fold]=roc_auc
        
        predictions[i:i+len
        (df)//k] = y_predict_test
        k_fold += 1
        print()
        
    #metricas globales
    acc_global = accuracy(df['target'], predictions)

    metricas["ACCURACY GLOBAL (validacion)"]= accuracy(df['target'], predictions)
    
    precision, recall, _ = precision_recall_curve(df['target'], predictions)
    metricas["AUPRC GLOBAL (Validacion)"]= auc(recall, precision)

    fpr, tpr, _ = roc_curve(df['target'], predictions)
    roc_auc = auc(fpr, tpr)
    metricas["AUCROC GLOBAL (validacion)"]=roc_auc


    #metricas promedio
    metricas["ACCURACY PROMEDIO (training)"] = metricas["ACCURACY (training)"].mean()
    metricas["ACCURACY PROMEDIO (validacion)"] = metricas["ACCURACY (validacion)"].mean()

    metricas["AUPRC PROMEDIO (training)"] = metricas["AUPRC (training)"].mean()
    metricas["AUPRC PROMEDIO (validacion)"] = metricas["AUPRC (validacion)"].mean()

    metricas["AUCROC PROMEDIO (training)"] = metricas["AUCROC (training)"].mean()
    metricas["AUCROC PROMEDIO (validacion)"] = metricas["AUCROC (validacion)"].mean()

    return metricas

In [49]:
metricas= k_fold(desarrollo,DecisionTreeClassifier,{"max_depth":3})
for key,values in metricas.items():

    print(key,values)

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251
positivos 23 negativos 67

fold:  4
entre 270 : 360
positivos 106 negativos 254
positivos 26 negativos 64

fold:  5
entre 360 : 450
positivos 108 negativos 252
positivos 24 negativos 66

ACCURACY (training) [0.85277778 0.85277778 0.83333333 0.83055556 0.85      ]
AUPRC (training) [0.77675753 0.77973422 0.76179803 0.74996696 0.78368056]
AUCROC (training) [0.82730769 0.7952381  0.78964874 0.74524588 0.80555556]
ACCURACY (validacion) [0.72222222 0.7        0.74444444 0.72222222 0.66666667]
AUPRC (validacion) [0.66937261 0.52962963 0.57198068 0.53222725 0.45833333]
AUCROC (validacion) [0.68642241 0.60582011 0.67131733 0.61057692 0.57386364]
ACCURACY GLOBAL (validacion) 0.7111111111111111
AUPRC GLOBAL (Validacion) 0.5563051568314725
AUCROC GLOBAL (validacion) 0.6316466552315609
ACCURAC

In [50]:
#EJ 2.3
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
grilla={"max_depth":[3,5,None],
        "criterion":["gini","entropy"],
       }

grilla_parametros=ParameterGrid(grilla)
performance={"Altura maxima":[],"Criterio de Corte":[],
             "Accuracy (training)":[],"Accuracy (validacion)":[]}
for params in grilla_parametros:
    res=k_fold(desarrollo,DecisionTreeClassifier,params,5)
    performance["Altura maxima"].append(params["max_depth"])
    performance["Criterio de Corte"].append(params["criterion"])
    performance["Accuracy (training)"].append(res["ACCURACY PROMEDIO (training)"])
    performance["Accuracy (validacion)"].append(res["ACCURACY PROMEDIO (validacion)"])

performance=pd.DataFrame(performance)
performance.fillna("infinito",inplace=True)
performance

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251
positivos 23 negativos 67

fold:  4
entre 270 : 360
positivos 106 negativos 254
positivos 26 negativos 64

fold:  5
entre 360 : 450
positivos 108 negativos 252
positivos 24 negativos 66

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251
positivos 23 negativos 67

fold:  4
entre 270 : 360
positivos 106 negativos 254
positivos 26 negativos 64

fold:  5
entre 360 : 450
positivos 108 negativos 252
positivos 24 negativos 66

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251

C:\Users\emili\AppData\Local\Temp\ipykernel_7576\252747801.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'infinito' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  performance.fillna("infinito",inplace=True)


,Altura maxima,Criterio de Corte,Accuracy (training),Accuracy (validacion)
0,3.0,gini,0.844444,0.706667
1,5.0,gini,0.945556,0.697778
2,infinito,gini,1.000000,0.673333
3,3.0,entropy,0.813889,0.693333
4,5.0,entropy,0.912778,0.688889
5,infinito,entropy,1.000000,0.675556


El arbol sobre ajusta y para gini tiene mejor performance

## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar: 
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo). 

2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default. 

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

3. ¿Cuál fue el mejor modelo y con qué configuración? Explicar por qué creería que dio mejor (recordando qué hace cada algoritmo y con qué tipo de datos están trabajando).

In [9]:
#EJ 3
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint, uniform
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

param_arbol = {
    "criterion": ["gini", "entropy"],          # categorías → lista normal
    "max_depth": randint(3, 20),               # enteros entre 3 y 19
    "max_features": uniform(0.1, 0.9),         # flotantes entre 0.1 y 1.0 (10%–100% de features)
    "min_samples_leaf": randint(1, 20),        # enteros entre 1 y 19
} #coctel anti overfit

param_KNN = {
    "algorithm":["ball_tree", "kd_tree", "brute"],
    "n_neighbors":randint(3, 20),
    "weights":["uniform", "distance"] #pesa los votos segun inversamente proporcional a las distancia
} #va fallar

param_SVM={
    "C":randint(1,300),
    "kernel":["linear", "poly", "rbf", "sigmoid"]
} 

arbol=DecisionTreeClassifier()
knn=KNeighborsClassifier()
svc=SVC()

random_search_arbol = RandomizedSearchCV(
    estimator=arbol,
    param_distributions=param_arbol,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=5,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_knn = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_KNN,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=5,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_svc = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_SVM,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=5,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

x_train = desarrollo.iloc[:,0:200].to_numpy()
y_train = desarrollo['target'].to_numpy()

random_search_arbol.fit(x_train,y_train)
random_search_knn.fit(x_train,y_train)
random_search_svc.fit(x_train,y_train)


RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000019E30418740>,
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42, scoring='roc_auc')

In [41]:
result_arbol=pd.DataFrame(random_search_arbol.cv_results_)
result_arbol=result_arbol.sort_values(by="mean_test_score",ascending=False)
result_arbol["criterion"] = result_arbol["params"].apply(lambda x: x["criterion"])
result_arbol["maxima profundidad"] = result_arbol["params"].apply(lambda x: x["max_depth"])
result_arbol["max_features"] = result_arbol["params"].apply(lambda x: x["max_features"])
result_arbol["min_samples_leaf"] = result_arbol["params"].apply(lambda x: x["min_samples_leaf"])
result_arbol=result_arbol[["criterion","maxima profundidad","max_features","min_samples_leaf","mean_test_score","std_test_score"]]
result_arbol.head(5)

,criterion,maxima profundidad,max_features,min_samples_leaf,mean_test_score,std_test_score
9,entropy,6,0.947982,14,0.724453,0.047813
5,gini,19,0.572297,10,0.704523,0.062287
4,entropy,3,0.373818,12,0.690893,0.016258
15,entropy,14,0.945549,4,0.668105,0.039124
11,entropy,17,0.714937,12,0.667215,0.025350


In [42]:
result_knn=pd.DataFrame(random_search_knn.cv_results_)
result_knn=result_knn.sort_values(by="mean_test_score",ascending=False)
result_knn["n_neighbors"]=result_knn["params"].apply(lambda x: x["n_neighbors"])
result_knn["algorithm"]=result_knn["params"].apply(lambda x: x["algorithm"])
result_knn["weights"]=result_knn["params"].apply(lambda x: x["weights"])
result_knn=result_knn[["n_neighbors","algorithm","weights","mean_test_score","std_test_score"]]
result_knn.head(5)

,n_neighbors,algorithm,weights,mean_test_score,std_test_score
14,16,ball_tree,distance,0.854296,0.033117
9,17,brute,distance,0.850868,0.033993
19,16,brute,uniform,0.843752,0.039010
15,11,kd_tree,distance,0.843111,0.042847
0,17,brute,uniform,0.840128,0.037122


In [40]:
result_svc=pd.DataFrame(random_search_svc.cv_results_)
result_svc=result_svc.sort_values(by="mean_test_score",ascending=False)
result_svc["C"]=result_svc["params"].apply(lambda x: x["C"])
result_svc["kernel"]=result_svc["params"].apply(lambda x: x["kernel"])
result_svc=result_svc[["C","kernel","mean_test_score","std_test_score"]]
result_svc.head(5)

,C,kernel,mean_test_score,std_test_score
1,271,rbf,0.89437,0.035144
3,21,rbf,0.89437,0.035144
5,215,rbf,0.89437,0.035144
4,122,rbf,0.89437,0.035144
16,49,rbf,0.89437,0.035144


In [51]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

resultado_lda=k_fold(desarrollo,LinearDiscriminantAnalysis,{})["AUCROC PROMEDIO (validacion)"]
resultado_bayes=k_fold(desarrollo,GaussianNB,{})["AUCROC PROMEDIO (validacion)"]
print("AUCROC PROMEDIO (validacion) LDA",resultado_lda)
print("AUCROC PROMEDIO (validacion) NAIVE BAYES",resultado_bayes)

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251
positivos 23 negativos 67

fold:  4
entre 270 : 360
positivos 106 negativos 254
positivos 26 negativos 64

fold:  5
entre 360 : 450
positivos 108 negativos 252
positivos 24 negativos 66

fold:  1
entre 0 : 90
positivos 100 negativos 260
positivos 32 negativos 58

fold:  2
entre 90 : 180
positivos 105 negativos 255
positivos 27 negativos 63

fold:  3
entre 180 : 270
positivos 109 negativos 251
positivos 23 negativos 67

fold:  4
entre 270 : 360
positivos 106 negativos 254
positivos 26 negativos 64

fold:  5
entre 360 : 450
positivos 108 negativos 252
positivos 24 negativos 66

AUCROC PROMEDIO (validacion) LDA 0.6472731602111527
AUCROC PROMEDIO (validacion) NAIVE BAYES 0.6995325975684328


## Ejercicio 4: 
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out). 

- Para ello, deberán predecir las **probabilidades** (probabilidad de la clase positiva) de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos. 

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`. 

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


## Ejercicio 5: 
### Conclusiones

Escribir como mínimo en un párrafo, una conclusión del trabajo realizado, incluyendo problemas encontrados y 
aspectos no incluidos en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio. 
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All). 
    - Las predicciones del *held out* del punto 4 en formato csv.
- Habrá una entrega intermedia obligatoria que deberán hacer antes de la fecha indicada en el campus. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios. 
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de 4 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.